In [1]:


# !python -m ipykernel install --user --name=tf_env --display-name "Python 3.11 (tf_env)"
!pip install matplotlib seaborn numpy pandas


In [38]:
import os
!pip install tensorflow

In [9]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [2]:
import tensorflow as tf
from tensorflow import keras

# Model building blocks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from IPython.display import Image, display
# Data preprocessing and augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Pre-trained model (our foundation)
from tensorflow.keras.applications import ResNet50

# Evaluation and visualization
from sklearn.metrics import (precision_recall_curve, roc_curve, accuracy_score,
                           confusion_matrix, precision_score, recall_score,
                           classification_report, roc_auc_score)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
tf.random.set_seed(42)
np.random.seed(42)

In [6]:
# Update this path to the actual location of your 'chest_xray' folder in Google Drive
base_dir = '/content/drive/My Drive/chest_xray'

train_dir = f'{base_dir}/train'
val_dir = f'{base_dir}/val'
test_dir = f'{base_dir}/test'

print(train_dir)
import os
print(os.path.exists(train_dir))

print(os.listdir(train_dir))  # Should show ['NORMAL', 'PNEUMONIA']


/content/drive/My Drive/chest_xray/train
True
['NORMAL', 'PNEUMONIA']


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
base_model = ResNet50(
    weights='imagenet',        # Use weights trained on ImageNet (1.4M images)
    include_top=False,         # Remove the final classification layer
    input_shape=(224, 224, 3)  # Our input: 224x224 RGB images
)

print(f"📊 ResNet-50 has {base_model.count_params():,} parameters")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
📊 ResNet-50 has 23,587,712 parameters


In [8]:
# Freeze all layers in the base model
base_model.trainable = False
trainable_count = sum([1 for layer in base_model.layers if layer.trainable])
total_count = len(base_model.layers)
print(f"📊 Trainable layers: {trainable_count}/{total_count}")

📊 Trainable layers: 0/175


In [9]:
model = Sequential([
    # 1. Feature Extraction (ResNet-50)
    base_model,

    # 2. Global Average Pooling
    GlobalAveragePooling2D(),

    # 3. Dense Layer for Learning
    Dense(256, activation='relu'),

    # 4. Dropout for Regularization
    Dropout(0.5),

    # 5. Output Layer
    Dense(1, activation='sigmoid')  # Binary classification: 0 or 1
])

In [10]:
model.summary()

total_params = np.sum([tf.keras.backend.count_params(w) for w in model.weights])

# trainable parameters
trainable_params = np.sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])

# non-trainable parameters
non_trainable_params = np.sum([tf.keras.backend.count_params(w) for w in model.non_trainable_weights])



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [11]:
if trainable_params < 1000000:
    print("Good! Small number of trainable parameters")
    print("   • Faster training")
    print("   • Less likely to overfit")
else:
    print("Large number of trainable parameters")
    print("   • Might need more data or regularization")

Good! Small number of trainable parameters
   • Faster training
   • Less likely to overfit


In [12]:
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Adam optimizer with learning rate
    loss='binary_crossentropy',           # Loss function for binary classification
    metrics=['accuracy']                  # Track accuracy during training
)


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ===============================
# 1️⃣ Create ImageDataGenerators
# ===============================

# Training data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation data: only rescale
val_datagen = ImageDataGenerator(rescale=1./255)

# ===============================
# 2️⃣ Create generators
# ===============================
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA'],
    shuffle=True,
    seed=42
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA'],
    shuffle=False
)

# ===============================
# 3️⃣ Print generator stats
# ===============================
print("Number of batches in train generator:", len(train_generator))
print("Batch size:", train_generator.batch_size)
print("Total training images:", train_generator.samples)


Found 5247 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Number of batches in train generator: 164
Batch size: 32
Total training images: 5247


In [30]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)


Found 624 images belonging to 2 classes.


In [14]:
import os

normal_count = len(os.listdir(f'{train_dir}/NORMAL'))
pneumonia_count = len(os.listdir(f'{train_dir}/PNEUMONIA'))
total_samples = normal_count + pneumonia_count

class_weight = {
    0: total_samples / (2.0 * normal_count),      # Weight for NORMAL
    1: total_samples / (2.0 * pneumonia_count)    # Weight for PNEUMONIA
}

print("NORMAL images:", normal_count)
print("PNEUMONIA images:", pneumonia_count)
print("Total training samples:", total_samples)
print("Class weights:", class_weight)


NORMAL images: 1341
PNEUMONIA images: 3906
Total training samples: 5247
Class weights: {0: 1.9563758389261745, 1: 0.6716589861751152}


In [15]:
print(f"   Normal:    {normal_count:,} images ({normal_count/total_samples*100:.1f}%)")
print(f"   Pneumonia: {pneumonia_count:,} images ({pneumonia_count/total_samples*100:.1f}%)")
print(f"   Total:     {total_samples:,} images")

print(f"\n⚖️ CALCULATED CLASS WEIGHTS:")
print(f"   Normal (0):    {class_weight[0]:.3f}")
print(f"   Pneumonia (1): {class_weight[1]:.3f}")
imbalance_ratio = pneumonia_count / normal_count
if imbalance_ratio > 2:
    print(f"⚠️  HIGH IMBALANCE DETECTED: {imbalance_ratio:.1f}:1 ratio")
    print("✅ Class weights will help balance this!")
else:
    print("✅ Moderate imbalance - class weights should handle this well")

   Normal:    1,341 images (25.6%)
   Pneumonia: 3,906 images (74.4%)
   Total:     5,247 images

⚖️ CALCULATED CLASS WEIGHTS:
   Normal (0):    1.956
   Pneumonia (1): 0.672
⚠️  HIGH IMBALANCE DETECTED: 2.9:1 ratio
✅ Class weights will help balance this!


In [16]:
early_stopping = EarlyStopping(
    monitor='val_loss',          # Watch validation loss
    patience=5,                  # Wait 5 epochs for improvement
    restore_best_weights=True,   # Keep the best weights, not the last
    verbose=1,                   # Print when it stops
    mode='min'                   # Lower validation loss is better
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',          # Watch validation loss
    factor=0.2,                  # Reduce LR by factor of 5 (multiply by 0.2)
    patience=3,                  # Wait 3 epochs before reducing
    min_lr=1e-7,                 # Don't go below this learning rate
    verbose=1,                   # Print when LR changes
    mode='min'                   # Lower loss is better
)
checkpoint = ModelCheckpoint(
    filepath='best_pneumonia_model.keras',
    monitor='val_accuracy',      # Save when validation accuracy improves
    save_best_only=True,         # Only save when performance improves
    verbose=1,                   # Print when saving
    mode='max'                   # Higher accuracy is better
)


In [17]:
callbacks = [early_stopping, reduce_lr, checkpoint]


In [18]:
EPOCHS = 15  # Start with 15 epochs
VERBOSE = 1  # Show progress bar
steps_per_epoch = len(train_generator)
estimated_minutes = (steps_per_epoch * EPOCHS * 3) / 60
print(f"\n⏱️ ESTIMATED TRAINING TIME:")
print(f"🔹 Steps per epoch: {steps_per_epoch}")
print(f"🔹 Estimated time: {estimated_minutes:.0f}-{estimated_minutes*2:.0f} minutes")
print(f"🔹 Note: Actual time depends on your hardware (GPU vs CPU)")


⏱️ ESTIMATED TRAINING TIME:
🔹 Steps per epoch: 164
🔹 Estimated time: 123-246 minutes
🔹 Note: Actual time depends on your hardware (GPU vs CPU)


In [19]:
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [22]:


try:
    batch_x, batch_y = next(train_generator)
    print(f"✅ Training generator working - batch shape: {batch_x.shape}")
    train_generator.reset()  # Reset generator
except:
    print("❌ Training generator issue!")

try:
    batch_x, batch_y = next(val_generator)
    print(f"✅ Validation generator working - batch shape: {batch_x.shape}")
    val_generator.reset()  # Reset generator
except:
    print("❌ Validation generator issue!")

if model.optimizer:
    print("✅ Model compiled successfully")
else:
    print("❌ Model not compiled!")





✅ Training generator working - batch shape: (32, 224, 224, 3)
✅ Validation generator working - batch shape: (16, 224, 224, 3)
🧠 Model Status:
✅ Model compiled successfully


In [23]:
istory = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    class_weight=class_weight,
    verbose=1
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 16s/step - accuracy: 0.5479 - loss: 0.7153 
Epoch 1: val_accuracy improved from -inf to 0.56250, saving model to best_pneumonia_model.keras
164/164 ━━━━━━━━━━━━━━━━━━━━ 2638s 16s/step - accuracy: 0.5481 - loss: 0.7152 - val_accuracy: 0.5625 - val_loss: 0.6605 - learning_rate: 0.0010
Epoch 2/15
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 726ms/step - accuracy: 0.6596 - loss: 0.6389
Epoch 2: val_accuracy improved from 0.56250 to 0.75000, saving model to best_pneumonia_model.keras
164/164 ━━━━━━━━━━━━━━━━━━━━ 151s 734ms/step - accuracy: 0.6596 - loss: 0.6389 - val_accuracy: 0.7500 - val_loss: 0.6129 - learning_rate: 0.0010
Epoch 3/15
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 716ms/step - accuracy: 0.7154 - loss: 0.5838
Epoch 3: val_accuracy did not improve from 0.75000
164/164 ━━━━━━━━━━━━━━━━━━━━ 118s 717ms/step - accuracy: 0.7154 - loss: 0.5838 - val_accuracy: 0.5625 - val_loss: 0.6257 - learning_rate: 0.0010
Epoch 4/15
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 723ms/step - a

In [25]:
# Unfreeze the last few layers of ResNet-50
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Keep early layers frozen
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Lower learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)




In [26]:
# Continue training
history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=callbacks,
    class_weight=class_weight,
    verbose=1
)

Epoch 1/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 783ms/step - accuracy: 0.7041 - loss: 0.6130
Epoch 1: val_accuracy did not improve from 0.75000
164/164 ━━━━━━━━━━━━━━━━━━━━ 152s 815ms/step - accuracy: 0.7044 - loss: 0.6121 - val_accuracy: 0.5625 - val_loss: 0.6528 - learning_rate: 1.0000e-04
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 747ms/step - accuracy: 0.8291 - loss: 0.3855
Epoch 2: val_accuracy did not improve from 0.75000
164/164 ━━━━━━━━━━━━━━━━━━━━ 183s 750ms/step - accuracy: 0.8291 - loss: 0.3854 - val_accuracy: 0.6875 - val_loss: 0.5602 - learning_rate: 1.0000e-04
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 745ms/step - accuracy: 0.8626 - loss: 0.3176
Epoch 3: val_accuracy did not improve from 0.75000
164/164 ━━━━━━━━━━━━━━━━━━━━ 122s 747ms/step - accuracy: 0.8626 - loss: 0.3176 - val_accuracy: 0.7500 - val_loss: 0.5844 - learning_rate: 1.0000e-04
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 756ms/step - accuracy: 0.8748 - loss: 0.2967
Epoch 4: val_accuracy did not improve from 0.750

In [31]:
print(f"\n📊 Evaluating model on test set...")

# Get test predictions
test_generator.reset()
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype(int).flatten()

# Get true labels
true_labels = test_generator.classes

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_classes)
precision = precision_score(true_labels, predicted_classes)
recall = recall_score(true_labels, predicted_classes)
auc = roc_auc_score(true_labels, predictions)

print(f"\n🎯 TEST RESULTS:")
print("=" * 30)
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"AUC-ROC:   {auc:.4f}")

# Classification report
print(f"\n📋 Detailed Classification Report:")
print(classification_report(true_labels, predicted_classes,
                          target_names=['Normal', 'Pneumonia']))


📊 Evaluating model on test set...
20/20 ━━━━━━━━━━━━━━━━━━━━ 206s 11s/step

🎯 TEST RESULTS:
Accuracy:  0.8253
Precision: 0.8808
Recall:    0.8333
AUC-ROC:   0.8923

📋 Detailed Classification Report:
              precision    recall  f1-score   support

      Normal       0.75      0.81      0.78       234
   Pneumonia       0.88      0.83      0.86       390

    accuracy                           0.83       624
   macro avg       0.81      0.82      0.82       624
weighted avg       0.83      0.83      0.83       624



In [2]:
def plot_training_history(istory, history_fine=None):
    """Plot training and validation metrics"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Combine histories if fine-tuning was done
    if history_fine:
        acc = istory.history['accuracy'] + history_fine.history['accuracy']
        val_acc = istory.history['val_accuracy'] + history_fine.history['val_accuracy']
        loss = istory.history['loss'] + history_fine.history['loss']
        val_loss = istory.history['val_loss'] + history_fine.history['val_loss']
    else:
        acc = istory.history['accuracy']
        val_acc = istory.history['val_accuracy']
        loss = istory.history['loss']
        val_loss = istory.history['val_loss']

    epochs = range(1, len(acc) + 1)

    # Accuracy plot
    axes[0].plot(epochs, acc, 'bo-', label='Training Accuracy')
    axes[0].plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
    axes[0].set_title('Training and Validation Accuracy')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True)

    # Loss plot
    axes[1].plot(epochs, loss, 'bo-', label='Training Loss')
    axes[1].plot(epochs, val_loss, 'ro-', label='Validation Loss')
    axes[1].set_title('Training and Validation Loss')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    plt.show()


In [3]:
plot_training_history(istory, history_fine)

NameError: name 'istory' is not defined